<a href="https://colab.research.google.com/github/jvataidee/PesquisaOperacional/blob/master/agregacao_sintese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# **Métodos de Agregação e Síntese**

---



In [ ]:
#importar bibliotecas
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as st
import scipy.stats as stats
from scipy.stats import chi2_contingency, spearmanr

# configurar a visualização
sns.set()
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

## **Método de Borda**
